## Signal, Images, and Video Project: Substituting Closed Eyes with Open Eyes in Images using DCGAN

In this project, we aim to substitute closed eyes in images with open eyes and open eyes in images with closed eyes using a Deep Convolutional Generative Adversarial Network (DCGAN) architecture. The input to the project consists of 5000 sample images of faces manually divided into opened and closed eyes taken from the Flickr-Faces-HQ Dataset (FFHQ) (https://github.com/NVlabs/ffhq-dataset).

A DCGAN is a type of generative model that is designed to generate new data that is similar to a training dataset. In this project, the DCGAN will be trained on the 5000 sample images of faces with either open or closed eyes to generate new images of faces with substituted eyes. The DCGAN architecture consists of two main components: a generator and a discriminator. The generator is trained to generate new images that are similar to the training data, while the discriminator is trained to determine whether an image is real or generated.

The DCGAN architecture is defined using the Keras library and consists of Convolutional Neural Networks (CNNs). The generator and discriminator models are first defined separately and then compiled into a combined model. The combined model is trained using the `fit()` function from Keras, with the training data, batch size, number of epochs, and other parameters specified.

The expected output of the project is a set of generated images of faces with substituted eyes. The quality of the generated images will be evaluated based on factors such as similarity to the training data and the sharpness of the generated images. The results will be displayed and saved as image files.

This project provides a unique opportunity to explore the use of DCGANs in image generation tasks and to gain insights into the capabilities and limitations of this type of model. With the expected results, we aim to demonstrate the potential of DCGANs in generating high-quality images of faces with substituted eyes.

## 1. Load, pre-process and split into training and validation data
The method `load_data_gen` loads and prepares a dataset for training and validation of a machine learning model. The method takes three inputs, the paths to the two directories containing the data of interest (open eyes and closed eyes), and the batch size. Firstly, the filenames and labels of the data are collected and stored in separate lists. The labels represent the binary classes of open eyes and closed eyes. The data is then shuffled to ensure that the model is not trained on ordered data. The shuffled data is then split into training and validation sets.

The method then defines a generator function, which takes in the filenames and labels, and yields the image data and corresponding label for each iteration. The generator is then used to create two datasets, one for the training set and one for the validation set. These datasets are created using the tf.data.Dataset.from_generator method, which converts the generator into a TensorFlow dataset. The datasets are then batched to the specified batch size. Finally, the prepared training and validation datasets are returned.

In [8]:
import numpy as np
import tensorflow as tf
from PIL import Image
from typing import Tuple
import os

def load_data_gen(path_open_eyes: str, path_closed_eyes: str, batch_size: int) -> Tuple:
	filenames = []
	labels = []

	for filename in os.listdir(path_open_eyes):
		if filename.endswith(".ini"):
			continue
		filenames.append(os.path.join(path_open_eyes, filename))
		labels.append([1.0, 0.0])

	for filename in os.listdir(path_closed_eyes):
		if filename.endswith(".ini"):
			continue
		filenames.append(os.path.join(path_closed_eyes, filename))
		labels.append([0.0, 1.0])

	# shuffle the data
	idx = np.arange(len(filenames))
	np.random.shuffle(idx)
	filenames = [filenames[i] for i in idx]
	labels = [labels[i] for i in idx]

	# split the data into training and validation sets
	split = int(0.8 * len(filenames))
	filenames_train = filenames[:split]
	labels_train = labels[:split]
	filenames_val = filenames[split:]
	labels_val = labels[split:]

	def generator(filenames, labels):
		for filename, label in zip(filenames, labels):
			image = Image.open(filename)
			image = image.resize((128, 128))
			image = np.array(image, dtype=np.float32)
			image = (image - 127.5) / 127.5
			yield image, label

	# create the training and validation generators
	train_gen = generator(filenames_train, labels_train)
	val_gen = generator(filenames_val, labels_val)

	# create the training and validation datasets
	train_ds = tf.data.Dataset.from_generator(lambda: train_gen, (tf.float32, tf.float32), ((128, 128, 3), (2,)))
	train_ds = train_ds.batch(batch_size)
	val_ds = tf.data.Dataset.from_generator(lambda: val_gen, (tf.float32, tf.float32), ((128, 128, 3), (2,)))
	val_ds = val_ds.batch(batch_size)

	return train_ds, val_ds


batch_size = 64
train_ds, val_ds = load_data_gen("dataset_1024/eyes_open/", "dataset_1024/eyes_closed/", batch_size)

## 2 - Build the generator and discriminator of the GAN
The code defines two functions, `build_generator` and `build_discriminator`, which return instances of the `Sequential` model from Tensorflow's Keras library. The build_generator function takes in a `latent_dim` parameter and returns a `Sequential` model that generates an image. The `build_discriminator` function takes in an `img_shape` parameter, which is the shape of the input image, and returns a `Sequential` model that discriminates between real and generated images.

The `build_generator` function starts by creating a `Sequential` model and adding a `Dense` layer with 128 * 8 * 8 units, which is reshaped into an 8 x 8 x 128 tensor. This tensor goes through a series of `Conv2DTranspose` layers, each with a `BatchNormalization` layer followed by a `ReLU` activation layer, to increase the spatial dimensions of the tensor. The final layer is a `Conv2DTranspose` layer with 3 filters and a `tanh` activation function.

The `build_discriminator` function starts by creating a `Sequential` model and adding a series of `Conv2D` layers with `LeakyReLU` activation functions, each followed by a `BatchNormalization` layer. The spatial dimensions of the input tensor are reduced with each `Conv2D` layer, with the final layer having a single output unit and a `tanh` activation function. This output is used to determine whether the input image is real or generated.

In [9]:
from keras.layers import Conv2D, LeakyReLU, Conv2DTranspose, ReLU, Activation, Dense, Flatten, Reshape, BatchNormalization
from keras.models import Model, Sequential
from typing import Tuple

def build_generator(latent_dim:int) -> Model:
	model = Sequential()
	model.add(Dense(128 * 128 * 3, activation="relu", input_dim=latent_dim))
	model.add(Reshape((128, 128, 3)))
	model.add(Conv2DTranspose(128, kernel_size=4, strides=1, padding='same'))
	model.add(BatchNormalization())
	model.add(ReLU())
	model.add(Conv2DTranspose(64, kernel_size=4, strides=1, padding='same'))
	model.add(BatchNormalization())
	model.add(ReLU())
	model.add(Conv2DTranspose(32, kernel_size=4, strides=1, padding='same'))
	model.add(BatchNormalization())
	model.add(ReLU())
	model.add(Conv2DTranspose(3, kernel_size=4, strides=1, padding='same'))
	model.add(Activation('tanh'))
	return model


def build_discriminator(img_shape:Tuple) -> Model:
	model = Sequential()
	model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Conv2D(64, kernel_size=3, strides=2, padding='same'))
	model.add(BatchNormalization())
	model.add(LeakyReLU(alpha=0.2))
	model.add(Conv2D(128, kernel_size=3, strides=2, padding='same'))
	model.add(BatchNormalization())
	model.add(LeakyReLU(alpha=0.2))
	model.add(Conv2D(256, kernel_size=3, strides=1, padding='same'))
	model.add(BatchNormalization())
	model.add(LeakyReLU(alpha=0.2))
	model.add(Flatten())
	# Use the tanh activation 
	model.add(Dense(1, activation='tanh'))
	return model

latent_dim = 100
img_shape = train_ds.element_spec[0].shape[1:]
generator = build_generator(latent_dim)
discriminator = build_discriminator(img_shape)

## Compile the models
The compile_models function takes the `generator` and `discriminator` as inputs and creates a combined model by connecting the two.
The `discriminator` is set to untrainable with `discriminator.trainable = False`, meaning its weights will not be updated during the training of the combined model.

The combined model takes in a `latent_dim`-shaped tensor as input and outputs the validity of the generated image as determined by the `discriminator`.
The combined model is then compiled with a binary cross-entropy loss function and the Adam optimizer.

In [12]:
from keras.layers import Input

def compile_models(generator:Model, discriminator:Model) -> Model:
	# set to False so that the weights of the discriminator are not updated during the training of the combined model
	discriminator.trainable = False

	z = Input(shape=(latent_dim,))
	img = generator(z)

	valid = discriminator(img)

	combined = Model(z, valid)
	combined.compile(loss="binary_crossentropy", optimizer="adam")
	return combined

combined = compile_models(generator, discriminator)

## Train the models
Train the generator and discriminator models in a GAN framework, where the generator tries to generate images that the discriminator cannot differentiate from real images, and the discriminator tries to correctly identify whether an image is real or generated.

In [14]:
from tqdm import tqdm

def train(combined:Model, dataset:tf.data.Dataset, latent_dim:int, epochs:int):
	for epoch in tqdm(range(epochs), desc="Epochs"):
		for x_batch in tqdm(dataset, desc="Training in batches"):
			valid = np.ones((batch_size, 1))
			fake = np.zeros((batch_size, 1))

			# train the discriminator to classify real and fake images
			imgs = x_batch
			z = np.random.normal(0, 1, (batch_size, latent_dim))
			gen_imgs = generator.predict(z)
			d_loss_real = discriminator.train_on_batch(imgs, valid)
			d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
			d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

			# train the generator to produce images that can fool the discriminator
			z = np.random.normal(0, 1, (batch_size, latent_dim))
			g_loss = combined.train_on_batch(z, valid)

			# print the losses
			print("%d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100 * d_loss[1], g_loss))

train(combined, train_ds, latent_dim, 100)

Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

2/2 [==============================] - 0s 60ms/step


Training in batches: 0it [00:05, ?it/s]
Epochs:   0%|          | 0/100 [00:05<?, ?it/s]


RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

## Generate new images
After training, use the generator model to generate new images with closed eyes open.

In [11]:
def generate_images(generator: Model, data: np.ndarray, n_images: int):
	predictions = generator.predict(data)
	for i in range(n_images):
		plt.imshow(predictions[i, :, :, 0], cmap='gray')
		plt.show()

generate_images(generator, data, 10)


InvalidArgumentError: Graph execution error:

Detected at node 'sequential/dense/Relu' defined at (most recent call last):
    File "c:\Users\maxst\miniconda3\envs\siv\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\maxst\miniconda3\envs\siv\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\maxst\miniconda3\envs\siv\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\maxst\miniconda3\envs\siv\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "c:\Users\maxst\miniconda3\envs\siv\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\maxst\miniconda3\envs\siv\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\maxst\miniconda3\envs\siv\lib\async

## Display and save the results
Finally, display and save the results by plotting the generated images and the original images side-by-side

In [7]:
def display_results(data, generated_images):
    # plot the results
    plt.imshow(np.concatenate([data, generated_images], axis=1))
    plt.show()

display_results(data, generated_images)


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 4 dimension(s) and the array at index 1 has 0 dimension(s)